In [ ]:
import numpy as np
import pandas as pd
import igraph as ig
import vmlab

In [ ]:
from vmlab.models import fruit_model
from vmlab.processes import harvest
fruit_model = fruit_model.update_processes({'harvest': harvest.HarvestByQuality})
fruit_model.visualize()

In [ ]:
tree = pd.read_csv('../share/trees/branch_G3.csv')
# tree

In [ ]:
setup = vmlab.create_setup(
    model=fruit_model,
    tree=tree,
    start_date='2003-06-01',
    end_date='2004-06-01',
    setup_toml='../share/setup/fruit_model_branch.toml',
    current_cycle=3,
    input_vars={
        'topology': {
            'seed': 11,
        },
        'geometry__interpretation_freq': 15
    },
    output_vars={
        'topology': {
            'adjacency': 'day'
        },
        'environment': {
            'TM_day': 'day',
            'RH_day': 'day'
        },
        'carbon_allocation': {
            'DM_fruit': 'day',
            'DM_fleshpeel': 'day',
            'DM_flesh': 'day'
        },
        'harvest': {
            'ripeness_index': 'day',
            'nb_fruit_harvested': None
        },
        'growth': {
            'nb_leaf': None,
        },
        'fruit_quality': {
            'FM_fruit': 'day',
            'W_fleshpeel': 'day',
            'W_flesh': 'day',
            'transpiration_fruit': 'day',
            'osmotic_pressure_fruit': 'day',
            'turgor_pressure_fruit': 'day',
            'water_potential_fruit': 'day',
            'flux_xylem_phloem': 'day',
            'sucrose': 'day'
        }
    }
)

In [ ]:
ds_out = vmlab.run(setup, fruit_model, geometry=True)

In [ ]:
g = ig.Graph.Adjacency([row.tolist() for row in ds_out.topology__adjacency[-1].data.astype(np.int64)])
layout = g.layout_reingold_tilford(root=0.)
layout.rotate(-180)
ig.plot(g, layout=layout, bbox=(600, 300), **{
    'vertex_size': 1,
    'vertex_label_size': 10,
    'edge_arrow_width': 0.1,
    'vertex_label': [
        f'GU{idx}\nF:{int(ds_out.harvest__nb_fruit_harvested.data[idx])}\nL:{int(ds_out.growth__nb_leaf.data[idx])}' for idx in g.vs.indices
    ]
})

In [ ]:
# how can we select all variables for GUs with at leat one fruit ? 
ds = ds_out[['fruit_quality__FM_fruit', 'carbon_allocation__DM_fruit']].sel({
        'day': slice('2003-06-01', '2004-06-01')
    }).where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).rename_vars({
        'fruit_quality__FM_fruit': 'FM',
        'carbon_allocation__DM_fruit': 'DM',
    })
ds

In [ ]:
ds_out.carbon_allocation__DM_fruit.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)
#ds.DM.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.carbon_allocation__DM_fleshpeel.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.carbon_allocation__DM_flesh.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__FM_fruit.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)
#ds.FM.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__W_fleshpeel.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__W_flesh.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__transpiration_fruit.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.environment__TM_day.sel({'day': slice('2004-01-01', '2004-04-01')}).plot(col_wrap=6)

In [ ]:
ds_out.environment__RH_day.sel({'day': slice('2004-01-01', '2004-04-01')}).plot(col_wrap=6)

In [ ]:
ds_out.fruit_quality__flux_xylem_phloem.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__water_potential_fruit.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__osmotic_pressure_fruit.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__turgor_pressure_fruit.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__sucrose.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)

In [ ]:
ds_out.harvest__ripeness_index.where(ds_out.carbon_allocation__DM_fruit > 0, drop=True).plot(col='GU', col_wrap=6)